# Flu model 2024

In [1]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import pandas as pd
import json
import warnings


warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
index_to_province = {
    0: 'Hebei',
    1: 'Tibet',
    2: 'Hainan',
    3: 'Liaoning',
    4: 'Heilongjiang',
    5: 'Henan',
    6: 'Sichuan',
    7: 'Qinghai',
    8: 'Shandong',
    9: 'Guizhou',
    10: 'Xinjiang',
    11: 'Gansu',
    12: 'Guangdong',
    13: 'Shanghai',
    14: 'Hunan',
    15: 'Beijing',
    16: 'Yunnan',
    17: 'Shanxi',
    18: 'Hubei',
    19: 'Chongqing',
    20: 'Fujian',
    21: 'Ningxia',
    22: 'Tianjin',
    23: 'Shaanxi',
    24: 'Anhui',
    25: 'Jiangsu',
    26: 'Inner Mongolia',
    27: 'Zhejiang',
    28: 'Jiangxi',
    29: 'Guangxi',
    30: 'Jilin'
}

## 1. Data and params

In [3]:
#Baidu migration data
flow_matrix_number_3d = np.load("D:\\Research\\Project\\PROJECTS\\其他\\Flu\\flow_matrix_number_3d_2023.npy")

sliced_data1 = flow_matrix_number_3d[211:365]*50000
sliced_data2 = flow_matrix_number_3d[0:63]*50000
sliced_data = np.concatenate((sliced_data1, sliced_data2), axis=0)

daily_outflow_np = sliced_data.sum(axis=2)
daily_inflow_np = sliced_data.sum(axis=1)

diff_flow = daily_inflow_np-daily_outflow_np

diff_flow_sum = diff_flow.sum(axis=0)
daily_inflow_np.shape

daily_inflow = daily_inflow_np
daily_outflow = daily_outflow_np


In [9]:
#Initial parameter andn values
para = pd.read_excel("D:\\Research\\Project\\PROJECTS\\其他\\Flu\\初值.xlsx")


Nas = np.array(para['Polulation_<18']) 
Nbs = np.array(para['Polulation_18-64'] )
Ncs = np.array(para['Polulation_65+'] )


I0as = np.array(para['<18 cases'])
I0bs = np.array(para['18-64 cases'] )
I0cs = np.array(para['65+ cases'])

n1s = np.array(para['VC1']*0.01)
n2s = np.array(para['VC2']*0.01)
n3s = np.array(para['VC3']*0.01)

VES1 = 0.65
VES2 = 0.50
VES3 = 0.42


VEI1 = 0.8
VEI2 = 0.8
VEI3 = 0.8

VEC1 = 0.69
VEC2 = 0.49
VEC3 = 0.50

VEF1 = 0.31
VEF2 = 0.28
VEF3 = 0.26


beta1s = np.array(para['Betas'] )

p = 0.500 
kappa= 0.333  
sigma = 0.53  
delta = 0.83  
gamma_A = 0.25
gamma_I = 0.31 
q = 0.003      
f = 0.17      

vt = 60
days = 216

## 2. Model

In [5]:
def flu_model_with_migration(y, t, Na,Nb,Nc, beta1, beta2,beta3,beta12, beta21,beta31,beta13,beta32,beta23,\
                             sigma, delta, gamma_A, gamma_I, kappa, p, q, f, n1,n2,n3,VEI1, VES1, VEC1, VEF1, \
                             VEI2, VES2, VEC2, VEF2,VEI3, VES3, VEC3, VEF3,inflow, outflow,X1,X2,X3):
    S1a, E1a, I1a, A1a, R1a, C1a, F1a, V2a, E2a, I2a, A2a, R2a, C2a, F2a, Ia, Ca, Fa, \
    S1b, E1b, I1b, A1b, R1b, C1b, F1b, V2b, E2b, I2b, A2b, R2b, C2b, F2b, Ib, Cb, Fb, \
    S1c, E1c, I1c, A1c, R1c, C1c, F1c, V2c, E2c, I2c, A2c, R2c, C2c, F2c, Ic, Cc, Fc = y

               
    #group a
    dS1adt = - beta1 * S1a * ((I1a + kappa*A1a) + (I2a + kappa*A2a) * (1-VEI1)) / Na \
        - beta21 * S1a * ((I1b + kappa*A1b) + (I2b + kappa*A2b) * (1-VEI2)) / Na \
        - beta31 * S1a * ((I1c + kappa*A1c) + (I2c + kappa*A2c) * (1-VEI3)) / Na \
        + (inflow - outflow) * (S1a / Na) - X1/vt*Na
    dE1adt = beta1 * S1a * ((I1a + kappa*A1a) + (I2a + kappa*A2a) * (1-VEI1)) / Na \
        + beta21 * S1a * ((I1b + kappa*A1b) + (I2b + kappa*A2b) * (1-VEI2)) / Na \
        + beta31 * S1a * ((I1c + kappa*A1c) + (I2c + kappa*A2c) * (1-VEI3)) / Na \
         - sigma * E1a * (1-p) - delta * E1a * p \
        + (inflow - outflow) * (E1a / Na)
    
    dI1adt = sigma * E1a * (1-p) - gamma_I * I1a + (inflow - outflow) * (I1a / Na) 
    dA1adt = delta * E1a * p - gamma_A * A1a + (inflow - outflow) * (A1a / Na) 
    dC1adt = q * I1a 
    dF1adt = f * q * I1a 
    dR1adt = gamma_I * I1a + gamma_A * A1a + (inflow - outflow) * (R1a / Na) 
    
    dV2adt = - beta1 *(1-VES1)* V2a * ((I1a + kappa*A1a) + (I2a + kappa*A2a) * (1-VEI1)) / Na \
        - beta21 *(1-VES1)* V2a * ((I1b + kappa*A1b) + (I2b + kappa*A2b) * (1-VEI2)) / Na \
        - beta31 *(1-VES1)* V2a * ((I1c + kappa*A1c) + (I2c + kappa*A2c) * (1-VEI3)) / Na \
        + (inflow - outflow) * (V2a / Na) + X1/vt*Na
    dE2adt = beta1 *(1-VES1)*  V2a * ((I1a + kappa*A1a) + (I2a + kappa*A2a) * (1-VEI1)) / Na \
        + beta21 * (1-VES1)* V2a * ((I1b + kappa*A1b) + (I2b + kappa*A2b) * (1-VEI2)) / Na \
        + beta31 *(1-VES1)*  V2a * ((I1c + kappa*A1c) + (I2c + kappa*A2c) * (1-VEI3)) / Na \
        - sigma * E2a * (1-p) - delta * E2a * p \
        + (inflow - outflow) * (E2a / Na)
    dI2adt = sigma * E2a * (1-p) - gamma_I * I2a + (inflow - outflow) * (I2a / Na)
    dA2adt = delta * E2a * p - gamma_A * A2a + (inflow - outflow) * (A2a / Na)
    dC2adt = q * I2a * (1-VEC1)
    dF2adt = f * q * I2a* (1-VEC1) * (1-VEF1)
    dR2adt = gamma_I * I2a + gamma_A * A2a + (inflow - outflow) * (R2a / Na)
    
    dIadt = sigma * E1a * (1-p) + delta * E1a * p  + sigma * E2a * (1-p)  + delta * E2a * p
    dCadt = q * I1a + q * I2a * (1-VEC1)
    dFadt = f * q * I1a + f * q * I2a * (1-VEC1) * (1-VEF1)
   
    
    # Group b
    dS1bdt = - beta2 * S1b * ((I1b + kappa*A1b) + (I2b + kappa*A2b) * (1-VEI2)) / Nb \
        - beta12 * S1b * ((I1a + kappa*A1a) + (I2a + kappa*A2a) * (1-VEI1)) / Nb \
        - beta32 * S1b * ((I1c + kappa*A1c) + (I2c + kappa*A2c) * (1-VEI3)) / Nb \
        + (inflow - outflow) * (S1b / Nb)- X2/vt*Nb
    dE1bdt = beta2 * S1b * ((I1b + kappa*A1b) + (I2b + kappa*A2b) * (1-VEI2)) / Nb \
        + beta12 * S1b * ((I1a + kappa*A1a) + (I2a + kappa*A2a) * (1-VEI1)) / Nb \
        + beta32 * S1b * ((I1c + kappa*A1c) + (I2c + kappa*A2c) * (1-VEI3)) / Nb \
          - sigma * E1b * (1-p) - delta * E1b * p \
        + (inflow - outflow) * (E1b / Nb)
    dI1bdt = sigma * E1b * (1-p) - gamma_I * I1b + (inflow - outflow) * (I1b / Nb)
    dA1bdt = delta * E1b * p - gamma_A * A1b + (inflow - outflow) * (A1b / Nb)
    dC1bdt = q * I1b
    dF1bdt = f * q * I1b
    dR1bdt = gamma_I * I1b + gamma_A * A1b + (inflow - outflow) * (R1b / Nb)
    
    dV2bdt = - beta2 * (1-VES2)*V2b * ((I1b + kappa*A1b) + (I2b + kappa*A2b) * (1-VEI2)) / Nb \
        - beta12*(1-VES2)* V2b * ((I1a + kappa*A1a) + (I2a + kappa*A2a) * (1-VEI1)) / Nb \
        - beta32 *(1-VES2)* V2b * ((I1c + kappa*A1c) + (I2c + kappa*A2c) * (1-VEI3)) / Nb \
        + (inflow - outflow) * (V2b / Nb)+ X2/vt*Nb
    dE2bdt = beta2 *(1-VES2)* V2b * ((I1b + kappa*A1b) + (I2b + kappa*A2b) * (1-VEI2)) / Nb \
        + beta12 *(1-VES2)* V2b * ((I1a + kappa*A1a) + (I2a + kappa*A2a) * (1-VEI1)) / Nb \
        + beta32 *(1-VES2)* V2b * ((I1c + kappa*A1c) + (I2c + kappa*A2c) * (1-VEI3)) / Nb \
          - sigma * E2b * (1-p) - delta * E2b * p \
        + (inflow - outflow) * (E2b / Nb)
    dI2bdt = sigma * E2b * (1-p) - gamma_I * I2b + (inflow - outflow) * (I2b / Nb)
    dA2bdt = delta * E2b * p - gamma_A * A2b + (inflow - outflow) * (A2b / Nb)
    dC2bdt = q * I2b * (1-VEC2)
    dF2bdt = f * q * I2b  * (1-VEC2)* (1-VEF2)
    dR2bdt = gamma_I * I2b + gamma_A * A2b + (inflow - outflow) * (R2b / Nb)
    
    dIbdt = sigma * E1b * (1-p) + delta * E1b * p+ delta * E2b * p  + sigma * E2b * (1-p)
    dCbdt = q * I1b + q * I2b * (1-VEC2)
    dFbdt = f * q * I1b + f * q * I2b * (1-VEC2) * (1-VEF2)
    
    # Group c
    dS1cdt = - beta3 * S1c * ((I1c + kappa*A1c) + (I2c + kappa*A2c) * (1-VEI3)) / Nc \
        - beta13 * S1c * ((I1a + kappa*A1a) + (I2a + kappa*A2a) * (1-VEI1)) / Nc \
        - beta23 * S1c * ((I1b + kappa*A1b) + (I2b + kappa*A2b) * (1-VEI2)) / Nc \
        + (inflow - outflow) * (S1c / Nc)- X3/vt*Nc
    dE1cdt = beta3 * S1c * ((I1c + kappa*A1c) + (I2c + kappa*A2c) * (1-VEI3)) / Nc \
        + beta13 * S1c * ((I1a + kappa*A1a) + (I2a + kappa*A2a) * (1-VEI1)) / Nc \
        + beta23 * S1c * ((I1b + kappa*A1b) + (I2b + kappa*A2b) * (1-VEI2)) / Nc \
         - sigma * E1c * (1-p) - delta * E1c * p \
        + (inflow - outflow) * (E1c / Nc)
    dI1cdt = sigma * E1c * (1-p) - gamma_I * I1c + (inflow - outflow) * (I1c / Nc)
    dA1cdt = delta * E1c * p - gamma_A * A1c + (inflow - outflow) * (A1c / Nc)
    dC1cdt = q * I1c
    dF1cdt = f * q * I1c
    dR1cdt = gamma_I * I1c + gamma_A * A1c + (inflow - outflow) * (R1c / Nc)
    
    
    dV2cdt = - beta3* (1-VES3)* V2c * ((I1c + kappa*A1c) + (I2c + kappa*A2c) * (1-VEI3)) / Nc \
        - beta13 *(1-VES3)* V2c * ((I1a + kappa*A1a) + (I2a + kappa*A2a) * (1-VEI1)) / Nc \
        - beta23 *(1-VES3)* V2c * ((I1b + kappa*A1b) + (I2b + kappa*A2b) * (1-VEI2)) / Nc \
        + (inflow - outflow) * (V2c / Nc)+ X3/vt*Nc
    dE2cdt = beta3 * (1-VES3)* V2c * ((I1c + kappa*A1c) + (I2c + kappa*A2c) * (1-VEI3)) / Nc \
        + beta13 *(1-VES3)* V2c * ((I1a + kappa*A1a) + (I2a + kappa*A2a) * (1-VEI1)) / Nc \
        + beta23 *(1-VES3)* V2c * ((I1b + kappa*A1b) + (I2b + kappa*A2b) * (1-VEI2)) / Nc \
        - sigma * E2c * (1-p) - delta * E2c * p \
        + (inflow - outflow) * (E2c / Nc)
    dI2cdt = sigma * E2c * (1-p) - gamma_I * I2c + (inflow - outflow) * (I2c / Nc)
    dA2cdt = delta * E2c * p - gamma_A * A2c + (inflow - outflow) * (A2c / Nc)
    dC2cdt = q * I2c * (1-VEC3)
    dF2cdt = f * q * I2c * (1-VEC3) * (1-VEF3)
    dR2cdt = gamma_I * I2c + gamma_A * A2c + (inflow - outflow) * (R2c / Nc)
    
    dIcdt = sigma * E1c * (1-p)  +delta * E1c * p+ sigma * E2c * (1-p) + delta * E2c * p 
    dCcdt = q * I1c + q * I2c * (1-VEC3)
    dFcdt = f * q * I1c + f * q * I2c* (1-VEC3) * (1-VEF3)

    Na = S1a+ E1a+ I1a+ A1a+ R1a+ C1a+ F1a+ V2a+ E2a+ I2a+ A2a+ R2a+ C2a+ F2a
    Nb= S1b+E1b+I1b+A1b+R1b+C1b+F1b+V2b+E2b+I2b+A2b+R2b+C2b+F2b
    Nc= S1c+E1c+I1c+A1c+R1c+C1c+F1c+V2c+E2c+I2c+A2c+R2c+C2c+F2c
    
    return dS1adt, dE1adt, dI1adt, dA1adt, dR1adt, dC1adt, dF1adt, \
        dV2adt, dE2adt, dI2adt, dA2adt, dR2adt, dC2adt, dF2adt, \
        dIadt, dCadt, dFadt, \
        dS1bdt, dE1bdt, dI1bdt, dA1bdt, dR1bdt, dC1bdt, dF1bdt, \
        dV2bdt, dE2bdt, dI2bdt, dA2bdt, dR2bdt, dC2bdt, dF2bdt,  \
        dIbdt, dCbdt, dFbdt, \
        dS1cdt, dE1cdt, dI1cdt, dA1cdt, dR1cdt, dC1cdt, dF1cdt, \
        dV2cdt, dE2cdt, dI2cdt, dA2cdt, dR2cdt, dC2cdt, dF2cdt, \
        dIcdt, dCcdt, dFcdt

## 3. Verification

In [7]:
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error, r2_score
##ILI+ and BMI data
ILI_plus = pd.read_excel("D:\\Research\\Project\\PROJECTS\\其他\\Flu\\ili.xlsx",index_col = 0)
BI = pd.read_excel("D:\\Research\\Project\\PROJECTS\\其他\\Flu\\BI_8M-2M.xlsx",index_col = 0)

In [11]:
loss_df = pd.DataFrame()
normalized_data_df = pd.DataFrame()
seiar_results_with_migration = []

for idx, (beta1,Na, Nb,Nc, I0xa, I0xb, I0xc,n1,n2,n3) in enumerate(zip(beta1s,Nas,Nbs,Ncs, I0as,I0bs,I0cs,n1s,n2s,n3s)):


    beta2 = beta1*0.94 
    beta3 = beta1*0.55
    beta12 = beta1*0.9
    beta32 =  beta1*0.5
    beta21 = beta1*0.9
    beta31 = beta1*0.5
    beta13 = beta1*0.9
    beta23 = beta1*0.9


    E1a0, I1a0, A1a0, R1a0, C1a0, F1a0, E2a0, I2a0, A2a0, R2a0, C2a0, F2a0 = 0, I0xa*(1-n1), 0, 0 , 0, 0 ,0, I0xa*n1, 0, 0 , 0, 0   
    I0a = I0xa
    C0a = 0
    F0a = 0
    S1a0 = Na*(1-n1) - E1a0 - I1a0 - A1a0 - R1a0 - C1a0 - F1a0
    V2a0 = Na*n1 - E2a0 - I2a0 - A2a0 - R2a0 - C2a0 - F2a0

    #Group b
    E1b0, I1b0, A1b0, R1b0, C1b0, F1b0, E2b0, I2b0, A2b0, R2b0, C2b0, F2b0 = 0, I0xb*(1-n2), 0, 0 , 0, 0 ,0, I0xb*n2, 0, 0 , 0, 0
    I0b = I0xb
    C0b = 0
    F0b = 0
    S1b0 = Nb*(1-n2) - E1b0 - I1b0 - A1b0 - R1b0 - C1b0 - F1b0
    V2b0 = Nb*n2 - E2b0 - I2b0 - A2b0 - R2b0 - C2b0 - F2b0

    #Group c
    E1c0, I1c0, A1c0, R1c0, C1c0, F1c0, E2c0, I2c0, A2c0, R2c0, C2c0, F2c0 = 0, I0xc*(1-n3), 0, 0 , 0, 0 ,0, I0xc*n3, 0, 0 , 0, 0
    I0c = I0xc
    C0c = 0
    F0c = 0
    S1c0 = Nc*(1-n3) - E1c0 - I1c0 - A1c0 - R1c0 - C1c0 - F1c0
    V2c0 = Nc*n3 - E2c0 - I2c0 - A2c0 - R2c0 - C2c0 - F2c0

    y0 =  S1a0, E1a0, I1a0, A1a0, R1a0, C1a0, F1a0, V2a0, E2a0, I2a0, A2a0, R2a0, C2a0, F2a0, I0a, C0a, F0a,  \
          S1b0, E1b0, I1b0, A1b0, R1b0, C1b0, F1b0, V2b0, E2b0, I2b0, A2b0, R2b0, C2b0, F2b0, I0b, C0b, F0b,  \
          S1c0, E1c0, I1c0, A1c0, R1c0, C1c0, F1c0, V2c0, E2c0, I2c0, A2c0, R2c0, C2c0, F2c0, I0c, C0c, F0c
    t = np.linspace(0, days, days+1)

    def X(t,vt):
        if t < vt: 
            return 0, 0, 0
        else:
            return 0, 0, 0
   
    def get_daily_migration(day, idx):
        if day < days:
            return daily_inflow[int(day), idx], daily_outflow[int(day), idx]
        else:
            return 0, 0  



    ret = odeint(lambda y, t: flu_model_with_migration(y, t, Na,Nb,Nc, beta1, beta2,beta3,beta12, beta21,beta31,beta13,beta32,beta23,
        sigma, delta, gamma_A, gamma_I, kappa, p, q, f, n1,n2,n3, VEI1,VES1, VEC1,VEF1,VEI2,VES2, VEC2,VEF2,VEI3,VES3, VEC3,VEF3,
        *get_daily_migration(t, idx),*X(t,vt) ),y0, t)



    S1a, E1a, I1a, A1a, R1a, C1a, F1a, V2a, E2a, I2a, A2a, R2a, C2a, F2a, Ia, Ca, Fa, \
    S1b, E1b, I1b, A1b, R1b, C1b, F1b, V2b, E2b, I2b, A2b, R2b, C2b, F2b, Ib, Cb, Fb, \
    S1c, E1c, I1c, A1c, R1c, C1c, F1c, V2c, E2c, I2c, A2c, R2c, C2c, F2c, Ic, Cc, Fc = ret.T


    S1a, E1a, I1a, A1a, R1a, C1a, F1a, V2a, E2a, I2a, A2a, R2a, C2a, F2a, Ia, Ca, Fa = S1a.tolist(), E1a.tolist(), I1a.tolist(), A1a.tolist(), R1a.tolist(), C1a.tolist(), F1a.tolist(), V2a.tolist(), E2a.tolist(), I2a.tolist(), A2a.tolist(), R2a.tolist(), C2a.tolist(), F2a.tolist(), Ia.tolist(), Ca.tolist(), Fa.tolist()
    S1b, E1b, I1b, A1b, R1b, C1b, F1b, V2b, E2b, I2b, A2b, R2b, C2b, F2b, Ib, Cb, Fb = S1b.tolist(), E1b.tolist(), I1b.tolist(), A1b.tolist(), R1b.tolist(), C1b.tolist(), F1b.tolist(), V2b.tolist(), E2b.tolist(), I2b.tolist(), A2b.tolist(), R2b.tolist(), C2b.tolist(), F2b.tolist(), Ib.tolist(), Cb.tolist(), Fb.tolist()
    S1c, E1c, I1c, A1c, R1c, C1c, F1c, V2c, E2c, I2c, A2c, R2c, C2c, F2c, Ic, Cc, Fc = S1c.tolist(), E1c.tolist(), I1c.tolist(), A1c.tolist(), R1c.tolist(), C1c.tolist(), F1c.tolist(), V2c.tolist(), E2c.tolist(), I2c.tolist(), A2c.tolist(), R2c.tolist(), C2c.tolist(), F2c.tolist(), Ic.tolist(), Cc.tolist(), Fc.tolist()
    seiar_results_with_migration.append({"Province": index_to_province[idx], 'Ia':Ia,'Ca' :Ca, 'Fa':Fa,'Ib':Ib,'Cb' :Cb, 'Fb':Fb, 'Ic':Ic,'Cc' :Cc, 'Fc':Fc})

for i in range(31):
    choose_number = i
    example_province_seiar_migration = seiar_results_with_migration[choose_number]
    province_name = index_to_province[choose_number].split('省')[0]


    y = [sum(x) for x in zip(seiar_results_with_migration[i]["Ia"], seiar_results_with_migration[i]["Ib"], seiar_results_with_migration[i]["Ic"])]
    diff_y = [y[i] - y[i-1] for i in range(1, len(y))]
    diff_y = [y[0]] + diff_y
    date_range = pd.date_range(start='2023-07-31', end='2024-03-03', freq='D')

    cases_df = pd.DataFrame(diff_y, index=date_range, columns=['Cases'])
    weekly_cases = cases_df.resample('W').sum()

    y2 = ILI_plus[province_name]

    scaler = MinMaxScaler(feature_range=(0, 1))

    
    weekly_cases_normalized = scaler.fit_transform(weekly_cases)
    y2_df = pd.DataFrame(y2)
    
    y2_normalized = scaler.fit_transform(y2_df)


    y3 = BI[BI['Province'] == province_name]['Daily_Cases'].resample('W').sum()
    y3_df = pd.DataFrame(y3)
    y3_normalized = scaler.fit_transform(y3_df)
    
    
    r2_12 = r2_score(weekly_cases_normalized, y2_normalized)
    r2_13 = r2_score(weekly_cases_normalized, y3_normalized)
    r2_23 = r2_score(y2_normalized, y3_normalized)
    
    mse12 = mean_squared_error(weekly_cases_normalized.flatten(), y2_normalized.flatten())
    mse13 = mean_squared_error(weekly_cases_normalized.flatten(), y3_normalized.flatten())
    mse23 = mean_squared_error(y2_normalized.flatten(), y3_normalized.flatten())
    
    rmse12 = np.sqrt(mse12)
    rmse13 = np.sqrt(mse13)
    rmse23 = np.sqrt(mse23)
    
    r12, p12 = pearsonr(weekly_cases_normalized.flatten(), y2_normalized.flatten())
    r13, p13 = pearsonr(weekly_cases_normalized.flatten(), y3_normalized.flatten())
    r23, p23 = pearsonr(y2_normalized.flatten(), y3_normalized.flatten())
    
    loss_df = loss_df.append({'province': province_name,'beta1 ':beta1, 'r12':r12, 'r13': r13, 'r23':r23,  'p12':p12, 'p13':p13, 'p23':p23,
                         'r2_12':r2_12,  'r2_13':r2_13,  'r2_23':r2_23,'mse12':mse12, 'mse13':mse13,'mse23':mse23,
                          'rmse12':rmse12,  'rmse13':rmse13, 'rmse23':rmse23}, ignore_index=True)
    



    normalized_data_df = pd.concat([normalized_data_df, pd.DataFrame(weekly_cases_normalized), pd.DataFrame(y2_normalized), pd.DataFrame(y3_normalized)], axis=1)
    normalized_data_df['province'] = province_name
loss_df.to_excel("D:\\Research\\Project\\PROJECTS\\其他\\Flu\\数据验证.xlsx")
normalized_data_df.to_excel("D:\\Research\\Project\\PROJECTS\\其他\\Flu\\标化数据.xlsx")

### National Verification

In [12]:
na = pd.read_csv("D:\\Research\\Project\\PROJECTS\\其他\\Flu\\national.csv",index_col = 0)

scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(na) 
std_df = scaler.transform(na)
std_df  = pd.DataFrame(std_df, columns= na.columns)

In [13]:
r2_12 = r2_score(std_df['cases'], std_df['ILI+'])
r2_13 = r2_score(std_df['cases'], std_df['index_baidu'])
r2_23 = r2_score(std_df['ILI+'], std_df['index_baidu'])

mse12 = mean_squared_error(std_df['cases'], std_df['ILI+'])
mse13 = mean_squared_error(std_df['cases'], std_df['index_baidu'])
mse23 = mean_squared_error(std_df['ILI+'], std_df['index_baidu'])

r12, p12 = pearsonr(std_df['cases'], std_df['ILI+'])
r13, p13 = pearsonr(std_df['cases'], std_df['index_baidu'])
r23, p23 = pearsonr(std_df['ILI+'], std_df['index_baidu'])

rmse12 = np.sqrt(mse12)
rmse13 = np.sqrt(mse13)
rmse23 = np.sqrt(mse23)

In [14]:
evaluation_df = pd.DataFrame({
    'Metric': ['r2_12', 'r2_13', 'r2_23', 'mse12', 'mse13', 'mse23', 'r12', 'r13', 'r23', 'p12', 'p13', 'p23','p112', 'p113', 'p123'],
    'Value': [r12, r13, r23, p12, p13, p23, r2_12, r2_13, r2_23, mse12, mse13, mse23,rmse12,rmse13,rmse23 ]
})

evaluation_df.to_excel("D:\\Research\\Project\\PROJECTS\\其他\\Flu\\全国数据验证指标.xlsx")

## 4. Scenarios 1-3

<b> Popualtion wide strategy, high risk population strategy, and age group vaccination strategy

In [15]:
seiar_results_with_migration = []
for rx in range(0,91):
    for idx, (beta1,Na, Nb,Nc, I0xa, I0xb, I0xc,n1,n2,n3) in enumerate(zip(beta1s,Nas,Nbs,Ncs, I0as,I0bs,I0cs,n1s,n2s,n3s)):
      
        n1 =  n1+ rx*0.01 
        n2 =  n2 + rx*0.01 
        n3 =  n3 + rx*0.01
       

        beta2 = beta1*0.94 
        beta3 = beta1*0.55
        beta12 = beta1*0.9
        beta32 =  beta1*0.5
        beta21 = beta1*0.9
        beta31 = beta1*0.5
        beta13 = beta1*0.9
        beta23 = beta1*0.9


        E1a0, I1a0, A1a0, R1a0, C1a0, F1a0, E2a0, I2a0, A2a0, R2a0, C2a0, F2a0 = 0, I0xa*(1-n1), 0, 0 , 0, 0 ,0, I0xa*n1, 0, 0 , 0, 0   
        I0a = I0xa
        C0a = 0
        F0a = 0
        S1a0 = Na*(1-n1) - E1a0 - I1a0 - A1a0 - R1a0 - C1a0 - F1a0
        V2a0 = Na*n1 - E2a0 - I2a0 - A2a0 - R2a0 - C2a0 - F2a0

        #Group b
        E1b0, I1b0, A1b0, R1b0, C1b0, F1b0, E2b0, I2b0, A2b0, R2b0, C2b0, F2b0 = 0, I0xb*(1-n2), 0, 0 , 0, 0 ,0, I0xb*n2, 0, 0 , 0, 0
        I0b = I0xb
        C0b = 0
        F0b = 0
        S1b0 = Nb*(1-n2) - E1b0 - I1b0 - A1b0 - R1b0 - C1b0 - F1b0
        V2b0 = Nb*n2 - E2b0 - I2b0 - A2b0 - R2b0 - C2b0 - F2b0

        #Group c
        E1c0, I1c0, A1c0, R1c0, C1c0, F1c0, E2c0, I2c0, A2c0, R2c0, C2c0, F2c0 = 0, I0xc*(1-n3), 0, 0 , 0, 0 ,0, I0xc*n3, 0, 0 , 0, 0
        I0c = I0xc
        C0c = 0
        F0c = 0
        S1c0 = Nc*(1-n3) - E1c0 - I1c0 - A1c0 - R1c0 - C1c0 - F1c0
        V2c0 = Nc*n3 - E2c0 - I2c0 - A2c0 - R2c0 - C2c0 - F2c0

        y0 =  S1a0, E1a0, I1a0, A1a0, R1a0, C1a0, F1a0, V2a0, E2a0, I2a0, A2a0, R2a0, C2a0, F2a0, I0a, C0a, F0a,  \
              S1b0, E1b0, I1b0, A1b0, R1b0, C1b0, F1b0, V2b0, E2b0, I2b0, A2b0, R2b0, C2b0, F2b0, I0b, C0b, F0b,  \
              S1c0, E1c0, I1c0, A1c0, R1c0, C1c0, F1c0, V2c0, E2c0, I2c0, A2c0, R2c0, C2c0, F2c0, I0c, C0c, F0c
        t = np.linspace(0, days, days+1) 

        def X(t,vt):
            if t < vt: 
                return 0, 0, 0
            else:
                return 0, 0, 0
 
        def get_daily_migration(day, idx):
            if day < days:
                return daily_inflow[int(day), idx], daily_outflow[int(day), idx]
            else:
                return 0, 0  
   
        ret = odeint(lambda y, t: flu_model_with_migration(y, t, Na,Nb,Nc, beta1, beta2,beta3,beta12, beta21,beta31,beta13,beta32,beta23,
            sigma, delta, gamma_A, gamma_I, kappa, p, q, f, n1,n2,n3, VEI1,VES1, VEC1,VEF1,VEI2,VES2, VEC2,VEF2,VEI3,VES3, VEC3,VEF3,
            *get_daily_migration(t, idx),*X(t,vt) ),y0, t)



        S1a, E1a, I1a, A1a, R1a, C1a, F1a, V2a, E2a, I2a, A2a, R2a, C2a, F2a, Ia, Ca, Fa, \
        S1b, E1b, I1b, A1b, R1b, C1b, F1b, V2b, E2b, I2b, A2b, R2b, C2b, F2b, Ib, Cb, Fb, \
        S1c, E1c, I1c, A1c, R1c, C1c, F1c, V2c, E2c, I2c, A2c, R2c, C2c, F2c, Ic, Cc, Fc = ret.T


        S1a, E1a, I1a, A1a, R1a, C1a, F1a, V2a, E2a, I2a, A2a, R2a, C2a, F2a, Ia, Ca, Fa = S1a.tolist(), E1a.tolist(), I1a.tolist(), A1a.tolist(), R1a.tolist(), C1a.tolist(), F1a.tolist(), V2a.tolist(), E2a.tolist(), I2a.tolist(), A2a.tolist(), R2a.tolist(), C2a.tolist(), F2a.tolist(), Ia.tolist(), Ca.tolist(), Fa.tolist()
        S1b, E1b, I1b, A1b, R1b, C1b, F1b, V2b, E2b, I2b, A2b, R2b, C2b, F2b, Ib, Cb, Fb = S1b.tolist(), E1b.tolist(), I1b.tolist(), A1b.tolist(), R1b.tolist(), C1b.tolist(), F1b.tolist(), V2b.tolist(), E2b.tolist(), I2b.tolist(), A2b.tolist(), R2b.tolist(), C2b.tolist(), F2b.tolist(), Ib.tolist(), Cb.tolist(), Fb.tolist()
        S1c, E1c, I1c, A1c, R1c, C1c, F1c, V2c, E2c, I2c, A2c, R2c, C2c, F2c, Ic, Cc, Fc = S1c.tolist(), E1c.tolist(), I1c.tolist(), A1c.tolist(), R1c.tolist(), C1c.tolist(), F1c.tolist(), V2c.tolist(), E2c.tolist(), I2c.tolist(), A2c.tolist(), R2c.tolist(), C2c.tolist(), F2c.tolist(), Ic.tolist(), Cc.tolist(), Fc.tolist()
        Ia_sum = Ia[-1]
        Ib_sum = Ib[-1]
        Ic_sum = Ic[-1]

        Ca_sum = Ca[-1]
        Cb_sum = Cb[-1]
        Cc_sum = Cc[-1]

        Fa_sum = Fa[-1]
        Fb_sum = Fb[-1]
        Fc_sum = Fc[-1]

        seiar_results_with_migration.append({'rx':rx,'n1':n1,'n2':n2,'n3':n3,'Province': index_to_province[idx],'未成年人累积病例数':Ia_sum,'中年人累积病例数' :Ib_sum, '老年人累积病例数':Ic_sum,
                                             '未成年人累积重症数':Ca_sum,'中年人累积重症数' :Cb_sum, '老年人累积重症数':Cc_sum,
                                             '未成年人累积死亡数':Fa_sum,'中年人累积死亡数' :Fb_sum, '老年人累积死亡数':Fc_sum,})
        
res_df = pd.DataFrame(seiar_results_with_migration)
res_df.to_excel('D:\\Research\\Project\\PROJECTS\\其他\\Flu\\高危人群策略.xlsx')

## 5. Sequential Immunization Strategies 

In [18]:
vts = [10,30,60]

seiar_results_with_migration1 = []
seiar_results_with_migration2 = []
seiar_results_with_migration3 = []

for goal in np.arange(0,0.91,0.01):
    for vt in vts:
        n1s = np.array(para['VC1']*0.01)

        n2s = np.array(para['VC2']*0.01)

        n3s = np.array(para['VC3']*0.01)

        if vt == 10:
            n1s = n1s + goal
            n2s = n2s + goal
            n3s = n3s + goal
            

            for idx, (beta1,Na, Nb,Nc, I0xa, I0xb, I0xc,n1,n2,n3) in enumerate(zip(beta1s,Nas,Nbs,Ncs, I0as,I0bs,I0cs,n1s,n2s,n3s)):

                beta2 = beta1*0.94 
                beta3 = beta1*0.55
                beta12 = beta1*0.9
                beta32 =  beta1*0.5
                beta21 = beta1*0.9
                beta31 = beta1*0.5
                beta13 = beta1*0.9
                beta23 = beta1*0.9


                E1a0, I1a0, A1a0, R1a0, C1a0, F1a0, E2a0, I2a0, A2a0, R2a0, C2a0, F2a0 = 0, I0xa*(1-n1), 0, 0 , 0, 0 ,0, I0xa*n1, 0, 0 , 0, 0   
                I0a = I0xa
                C0a = 0
                F0a = 0
                S1a0 = Na*(1-n1) - E1a0 - I1a0 - A1a0 - R1a0 - C1a0 - F1a0
                V2a0 = Na*n1 - E2a0 - I2a0 - A2a0 - R2a0 - C2a0 - F2a0

                #Group b
                E1b0, I1b0, A1b0, R1b0, C1b0, F1b0, E2b0, I2b0, A2b0, R2b0, C2b0, F2b0 = 0, I0xb*(1-n2), 0, 0 , 0, 0 ,0, I0xb*n2, 0, 0 , 0, 0
                I0b = I0xb
                C0b = 0
                F0b = 0
                S1b0 = Nb*(1-n2) - E1b0 - I1b0 - A1b0 - R1b0 - C1b0 - F1b0
                V2b0 = Nb*n2 - E2b0 - I2b0 - A2b0 - R2b0 - C2b0 - F2b0

                #Group c
                E1c0, I1c0, A1c0, R1c0, C1c0, F1c0, E2c0, I2c0, A2c0, R2c0, C2c0, F2c0 = 0, I0xc*(1-n3), 0, 0 , 0, 0 ,0, I0xc*n3, 0, 0 , 0, 0
                I0c = I0xc
                C0c = 0
                F0c = 0
                S1c0 = Nc*(1-n3) - E1c0 - I1c0 - A1c0 - R1c0 - C1c0 - F1c0
                V2c0 = Nc*n3 - E2c0 - I2c0 - A2c0 - R2c0 - C2c0 - F2c0

                y0 =  S1a0, E1a0, I1a0, A1a0, R1a0, C1a0, F1a0, V2a0, E2a0, I2a0, A2a0, R2a0, C2a0, F2a0, I0a, C0a, F0a,  \
                    S1b0, E1b0, I1b0, A1b0, R1b0, C1b0, F1b0, V2b0, E2b0, I2b0, A2b0, R2b0, C2b0, F2b0, I0b, C0b, F0b,  \
                    S1c0, E1c0, I1c0, A1c0, R1c0, C1c0, F1c0, V2c0, E2c0, I2c0, A2c0, R2c0, C2c0, F2c0, I0c, C0c, F0c
                t = np.linspace(0, days, days+1) 

                def X(t,vt):
                    if t < vt: 
                        return 0, 0, 0
                    else:
                        return 0, 0, 0
                
                def get_daily_migration(day, idx):
                    if day < days:
                        return daily_inflow[int(day), idx], daily_outflow[int(day), idx]
                    else:
                        return 0, 0  


                ret = odeint(lambda y, t: flu_model_with_migration(y, t, Na,Nb,Nc, beta1, beta2,beta3,beta12, beta21,beta31,beta13,beta32,beta23,
                    sigma, delta, gamma_A, gamma_I, kappa, p, q, f, n1,n2,n3, VEI1,VES1, VEC1,VEF1,VEI2,VES2, VEC2,VEF2,VEI3,VES3, VEC3,VEF3,
                    *get_daily_migration(t, idx),*X(t,vt) ),y0, t)



                S1a, E1a, I1a, A1a, R1a, C1a, F1a, V2a, E2a, I2a, A2a, R2a, C2a, F2a, Ia, Ca, Fa, \
                S1b, E1b, I1b, A1b, R1b, C1b, F1b, V2b, E2b, I2b, A2b, R2b, C2b, F2b, Ib, Cb, Fb, \
                S1c, E1c, I1c, A1c, R1c, C1c, F1c, V2c, E2c, I2c, A2c, R2c, C2c, F2c, Ic, Cc, Fc = ret.T


                S1a, E1a, I1a, A1a, R1a, C1a, F1a, V2a, E2a, I2a, A2a, R2a, C2a, F2a, Ia, Ca, Fa = S1a.tolist(), E1a.tolist(), I1a.tolist(), A1a.tolist(), R1a.tolist(), C1a.tolist(), F1a.tolist(), V2a.tolist(), E2a.tolist(), I2a.tolist(), A2a.tolist(), R2a.tolist(), C2a.tolist(), F2a.tolist(), Ia.tolist(), Ca.tolist(), Fa.tolist()
                S1b, E1b, I1b, A1b, R1b, C1b, F1b, V2b, E2b, I2b, A2b, R2b, C2b, F2b, Ib, Cb, Fb = S1b.tolist(), E1b.tolist(), I1b.tolist(), A1b.tolist(), R1b.tolist(), C1b.tolist(), F1b.tolist(), V2b.tolist(), E2b.tolist(), I2b.tolist(), A2b.tolist(), R2b.tolist(), C2b.tolist(), F2b.tolist(), Ib.tolist(), Cb.tolist(), Fb.tolist()
                S1c, E1c, I1c, A1c, R1c, C1c, F1c, V2c, E2c, I2c, A2c, R2c, C2c, F2c, Ic, Cc, Fc = S1c.tolist(), E1c.tolist(), I1c.tolist(), A1c.tolist(), R1c.tolist(), C1c.tolist(), F1c.tolist(), V2c.tolist(), E2c.tolist(), I2c.tolist(), A2c.tolist(), R2c.tolist(), C2c.tolist(), F2c.tolist(), Ic.tolist(), Cc.tolist(), Fc.tolist()

                seiar_results_with_migration1.append({"Province": index_to_province[idx],'goal':goal,'n1s':n1s,'n2s':n2s, 'n3s':n3s,'Ia':Ia,'Ca' :Ca, 'Fa':Fa,'Ib':Ib,'Cb' :Cb, 'Fb':Fb, 'Ic':Ic,'Cc' :Cc, 'Fc':Fc})

            infection_df = pd.DataFrame()

            for i in range(31):
                choose_number = i
                example_province_seiar_migration = seiar_results_with_migration1[choose_number]
                province_name = index_to_province[choose_number].split('省')[0]

                y1 = [sum(x) for x in zip(seiar_results_with_migration1[i]["Ia"], seiar_results_with_migration1[i]["Ib"], seiar_results_with_migration1[i]["Ic"])]
                diff_y1 = [y1[i] - y1[i-1] for i in range(1, len(y1))]
                diff_y1 = [y1[0]] + diff_y1
                date_range = pd.date_range(start='2023-07-31', end='2024-03-03', freq='D')

                y2 = [sum(x) for x in zip(seiar_results_with_migration1[i]["Ca"], seiar_results_with_migration1[i]["Cb"], seiar_results_with_migration1[i]["Cc"])]
                diff_y2 = [y2[i] - y2[i-1] for i in range(1, len(y2))]
                diff_y2 = [y2[0]] + diff_y2


                y3 = [sum(x) for x in zip(seiar_results_with_migration1[i]["Fa"], seiar_results_with_migration1[i]["Fb"], seiar_results_with_migration1[i]["Fc"])]
                diff_y3 = [y3[i] - y3[i-1] for i in range(1, len(y3))]
                diff_y3 = [y3[0]] + diff_y3

                for j in range(len(date_range)):
                    case = diff_y1[j]
                    zz = diff_y2[j]
                    death = diff_y3[j]

                    infection_df = infection_df.append({'date': date_range[j], 'province': province_name, '感染': case, '重症':zz,'死亡':death}, ignore_index=True)
                    infection_df = pd.concat([infection_df, pd.DataFrame({'date': date_range[j], 'province': province_name, '感染': case, '重症':zz,'死亡':death}, index=[0])], ignore_index=True)

            infection_df.to_excel(f"D:\\Research\\Project\\PROJECTS\\其他\\Flu\\延迟{vt-10}天_目标疫苗覆盖率{goal*100}%.xlsx")
        elif vt == 30:
            n1s = n1s + goal/2
            n2s = n2s + goal/2
            n3s = n3s + goal/2
            
            for idx, (beta1,Na, Nb,Nc, I0xa, I0xb, I0xc,n1,n2,n3) in enumerate(zip(beta1s,Nas,Nbs,Ncs, I0as,I0bs,I0cs,n1s,n2s,n3s)):

                beta2 = beta1*0.94 
                beta3 = beta1*0.55
                beta12 = beta1*0.9
                beta32 =  beta1*0.5
                beta21 = beta1*0.9
                beta31 = beta1*0.5
                beta13 = beta1*0.9
                beta23 = beta1*0.9


                E1a0, I1a0, A1a0, R1a0, C1a0, F1a0, E2a0, I2a0, A2a0, R2a0, C2a0, F2a0 = 0, I0xa*(1-n1), 0, 0 , 0, 0 ,0, I0xa*n1, 0, 0 , 0, 0   
                I0a = I0xa
                C0a = 0
                F0a = 0
                S1a0 = Na*(1-n1) - E1a0 - I1a0 - A1a0 - R1a0 - C1a0 - F1a0
                V2a0 = Na*n1 - E2a0 - I2a0 - A2a0 - R2a0 - C2a0 - F2a0

                #Group b
                E1b0, I1b0, A1b0, R1b0, C1b0, F1b0, E2b0, I2b0, A2b0, R2b0, C2b0, F2b0 = 0, I0xb*(1-n2), 0, 0 , 0, 0 ,0, I0xb*n2, 0, 0 , 0, 0
                I0b = I0xb
                C0b = 0
                F0b = 0
                S1b0 = Nb*(1-n2) - E1b0 - I1b0 - A1b0 - R1b0 - C1b0 - F1b0
                V2b0 = Nb*n2 - E2b0 - I2b0 - A2b0 - R2b0 - C2b0 - F2b0

                #Group c
                E1c0, I1c0, A1c0, R1c0, C1c0, F1c0, E2c0, I2c0, A2c0, R2c0, C2c0, F2c0 = 0, I0xc*(1-n3), 0, 0 , 0, 0 ,0, I0xc*n3, 0, 0 , 0, 0
                I0c = I0xc
                C0c = 0
                F0c = 0
                S1c0 = Nc*(1-n3) - E1c0 - I1c0 - A1c0 - R1c0 - C1c0 - F1c0
                V2c0 = Nc*n3 - E2c0 - I2c0 - A2c0 - R2c0 - C2c0 - F2c0

                y0 =  S1a0, E1a0, I1a0, A1a0, R1a0, C1a0, F1a0, V2a0, E2a0, I2a0, A2a0, R2a0, C2a0, F2a0, I0a, C0a, F0a,  \
                    S1b0, E1b0, I1b0, A1b0, R1b0, C1b0, F1b0, V2b0, E2b0, I2b0, A2b0, R2b0, C2b0, F2b0, I0b, C0b, F0b,  \
                    S1c0, E1c0, I1c0, A1c0, R1c0, C1c0, F1c0, V2c0, E2c0, I2c0, A2c0, R2c0, C2c0, F2c0, I0c, C0c, F0c
                t = np.linspace(0, days, days+1) 
                def X(t,vt):
                    if t < vt: 
                        return goal/2, goal/2, goal/2
                    else:
                        return 0, 0, 0
                
                def get_daily_migration(day, idx):
                    if day < days:
                        return daily_inflow[int(day), idx], daily_outflow[int(day), idx]
                    else:
                        return 0, 0 


                ret = odeint(lambda y, t: flu_model_with_migration(y, t, Na,Nb,Nc, beta1, beta2,beta3,beta12, beta21,beta31,beta13,beta32,beta23,
                    sigma, delta, gamma_A, gamma_I, kappa, p, q, f, n1,n2,n3, VEI1,VES1, VEC1,VEF1,VEI2,VES2, VEC2,VEF2,VEI3,VES3, VEC3,VEF3,
                    *get_daily_migration(t, idx),*X(t,vt) ),y0, t)



                S1a, E1a, I1a, A1a, R1a, C1a, F1a, V2a, E2a, I2a, A2a, R2a, C2a, F2a, Ia, Ca, Fa, \
                S1b, E1b, I1b, A1b, R1b, C1b, F1b, V2b, E2b, I2b, A2b, R2b, C2b, F2b, Ib, Cb, Fb, \
                S1c, E1c, I1c, A1c, R1c, C1c, F1c, V2c, E2c, I2c, A2c, R2c, C2c, F2c, Ic, Cc, Fc = ret.T


                S1a, E1a, I1a, A1a, R1a, C1a, F1a, V2a, E2a, I2a, A2a, R2a, C2a, F2a, Ia, Ca, Fa = S1a.tolist(), E1a.tolist(), I1a.tolist(), A1a.tolist(), R1a.tolist(), C1a.tolist(), F1a.tolist(), V2a.tolist(), E2a.tolist(), I2a.tolist(), A2a.tolist(), R2a.tolist(), C2a.tolist(), F2a.tolist(), Ia.tolist(), Ca.tolist(), Fa.tolist()
                S1b, E1b, I1b, A1b, R1b, C1b, F1b, V2b, E2b, I2b, A2b, R2b, C2b, F2b, Ib, Cb, Fb = S1b.tolist(), E1b.tolist(), I1b.tolist(), A1b.tolist(), R1b.tolist(), C1b.tolist(), F1b.tolist(), V2b.tolist(), E2b.tolist(), I2b.tolist(), A2b.tolist(), R2b.tolist(), C2b.tolist(), F2b.tolist(), Ib.tolist(), Cb.tolist(), Fb.tolist()
                S1c, E1c, I1c, A1c, R1c, C1c, F1c, V2c, E2c, I2c, A2c, R2c, C2c, F2c, Ic, Cc, Fc = S1c.tolist(), E1c.tolist(), I1c.tolist(), A1c.tolist(), R1c.tolist(), C1c.tolist(), F1c.tolist(), V2c.tolist(), E2c.tolist(), I2c.tolist(), A2c.tolist(), R2c.tolist(), C2c.tolist(), F2c.tolist(), Ic.tolist(), Cc.tolist(), Fc.tolist()

                seiar_results_with_migration2.append({"Province": index_to_province[idx],'goal':goal,'n1s':n1s,'n2s':n2s, 'n3s':n3s,'Ia':Ia,'Ca' :Ca, 'Fa':Fa,'Ib':Ib,'Cb' :Cb, 'Fb':Fb, 'Ic':Ic,'Cc' :Cc, 'Fc':Fc})

            infection_df = pd.DataFrame()

            for i in range(31):
                choose_number = i
                example_province_seiar_migration = seiar_results_with_migration2[choose_number]
                province_name = index_to_province[choose_number].split('省')[0]

                y1 = [sum(x) for x in zip(seiar_results_with_migration2[i]["Ia"], seiar_results_with_migration2[i]["Ib"], seiar_results_with_migration2[i]["Ic"])]
                diff_y1 = [y1[i] - y1[i-1] for i in range(1, len(y1))]
                diff_y1 = [y1[0]] + diff_y1
                date_range = pd.date_range(start='2023-07-31', end='2024-03-03', freq='D')

                y2 = [sum(x) for x in zip(seiar_results_with_migration2[i]["Ca"], seiar_results_with_migration2[i]["Cb"], seiar_results_with_migration2[i]["Cc"])]
                diff_y2 = [y2[i] - y2[i-1] for i in range(1, len(y2))]
                diff_y2 = [y2[0]] + diff_y2


                y3 = [sum(x) for x in zip(seiar_results_with_migration2[i]["Fa"], seiar_results_with_migration2[i]["Fb"], seiar_results_with_migration2[i]["Fc"])]
                diff_y3 = [y3[i] - y3[i-1] for i in range(1, len(y3))]
                diff_y3 = [y3[0]] + diff_y3

                for j in range(len(date_range)):
                    case = diff_y1[j]
                    zz = diff_y2[j]
                    death = diff_y3[j]

                  
                    infection_df = pd.concat([infection_df, pd.DataFrame({'date': date_range[j], 'province': province_name, '感染': case, '重症':zz,'死亡':death}, index=[0])], ignore_index=True)

            infection_df.to_excel(f"D:\\Research\\Project\\PROJECTS\\其他\\Flu\\延迟{vt}天_目标疫苗覆盖率{goal*100}%.xlsx")
        else:
            n1s = n1s
            n2s = n2s
            n3s = n3s
         
            for idx, (beta1,Na, Nb,Nc, I0xa, I0xb, I0xc,n1,n2,n3) in enumerate(zip(beta1s,Nas,Nbs,Ncs, I0as,I0bs,I0cs,n1s,n2s,n3s)):

                beta2 = beta1*0.94 
                beta3 = beta1*0.55
                beta12 = beta1*0.9
                beta32 =  beta1*0.5
                beta21 = beta1*0.9
                beta31 = beta1*0.5
                beta13 = beta1*0.9
                beta23 = beta1*0.9


                E1a0, I1a0, A1a0, R1a0, C1a0, F1a0, E2a0, I2a0, A2a0, R2a0, C2a0, F2a0 = 0, I0xa*(1-n1), 0, 0 , 0, 0 ,0, I0xa*n1, 0, 0 , 0, 0   
                I0a = I0xa
                C0a = 0
                F0a = 0
                S1a0 = Na*(1-n1) - E1a0 - I1a0 - A1a0 - R1a0 - C1a0 - F1a0
                V2a0 = Na*n1 - E2a0 - I2a0 - A2a0 - R2a0 - C2a0 - F2a0

                #Group b
                E1b0, I1b0, A1b0, R1b0, C1b0, F1b0, E2b0, I2b0, A2b0, R2b0, C2b0, F2b0 = 0, I0xb*(1-n2), 0, 0 , 0, 0 ,0, I0xb*n2, 0, 0 , 0, 0
                I0b = I0xb
                C0b = 0
                F0b = 0
                S1b0 = Nb*(1-n2) - E1b0 - I1b0 - A1b0 - R1b0 - C1b0 - F1b0
                V2b0 = Nb*n2 - E2b0 - I2b0 - A2b0 - R2b0 - C2b0 - F2b0

                #Group c
                E1c0, I1c0, A1c0, R1c0, C1c0, F1c0, E2c0, I2c0, A2c0, R2c0, C2c0, F2c0 = 0, I0xc*(1-n3), 0, 0 , 0, 0 ,0, I0xc*n3, 0, 0 , 0, 0
                I0c = I0xc
                C0c = 0
                F0c = 0
                S1c0 = Nc*(1-n3) - E1c0 - I1c0 - A1c0 - R1c0 - C1c0 - F1c0
                V2c0 = Nc*n3 - E2c0 - I2c0 - A2c0 - R2c0 - C2c0 - F2c0

                y0 =  S1a0, E1a0, I1a0, A1a0, R1a0, C1a0, F1a0, V2a0, E2a0, I2a0, A2a0, R2a0, C2a0, F2a0, I0a, C0a, F0a,  \
                    S1b0, E1b0, I1b0, A1b0, R1b0, C1b0, F1b0, V2b0, E2b0, I2b0, A2b0, R2b0, C2b0, F2b0, I0b, C0b, F0b,  \
                    S1c0, E1c0, I1c0, A1c0, R1c0, C1c0, F1c0, V2c0, E2c0, I2c0, A2c0, R2c0, C2c0, F2c0, I0c, C0c, F0c
                t = np.linspace(0, days, days+1) 

                def X(t,vt):
                    if t < vt: 
                        return goal, goal, goal
                    else:
                        return 0, 0, 0
              
                def get_daily_migration(day, idx):
                    if day < days:
                        return daily_inflow[int(day), idx], daily_outflow[int(day), idx]
                    else:
                        return 0, 0  


                ret = odeint(lambda y, t: flu_model_with_migration(y, t, Na,Nb,Nc, beta1, beta2,beta3,beta12, beta21,beta31,beta13,beta32,beta23,
                    sigma, delta, gamma_A, gamma_I, kappa, p, q, f, n1,n2,n3, VEI1,VES1, VEC1,VEF1,VEI2,VES2, VEC2,VEF2,VEI3,VES3, VEC3,VEF3,
                    *get_daily_migration(t, idx),*X(t,vt) ),y0, t)



                S1a, E1a, I1a, A1a, R1a, C1a, F1a, V2a, E2a, I2a, A2a, R2a, C2a, F2a, Ia, Ca, Fa, \
                S1b, E1b, I1b, A1b, R1b, C1b, F1b, V2b, E2b, I2b, A2b, R2b, C2b, F2b, Ib, Cb, Fb, \
                S1c, E1c, I1c, A1c, R1c, C1c, F1c, V2c, E2c, I2c, A2c, R2c, C2c, F2c, Ic, Cc, Fc = ret.T


                S1a, E1a, I1a, A1a, R1a, C1a, F1a, V2a, E2a, I2a, A2a, R2a, C2a, F2a, Ia, Ca, Fa = S1a.tolist(), E1a.tolist(), I1a.tolist(), A1a.tolist(), R1a.tolist(), C1a.tolist(), F1a.tolist(), V2a.tolist(), E2a.tolist(), I2a.tolist(), A2a.tolist(), R2a.tolist(), C2a.tolist(), F2a.tolist(), Ia.tolist(), Ca.tolist(), Fa.tolist()
                S1b, E1b, I1b, A1b, R1b, C1b, F1b, V2b, E2b, I2b, A2b, R2b, C2b, F2b, Ib, Cb, Fb = S1b.tolist(), E1b.tolist(), I1b.tolist(), A1b.tolist(), R1b.tolist(), C1b.tolist(), F1b.tolist(), V2b.tolist(), E2b.tolist(), I2b.tolist(), A2b.tolist(), R2b.tolist(), C2b.tolist(), F2b.tolist(), Ib.tolist(), Cb.tolist(), Fb.tolist()
                S1c, E1c, I1c, A1c, R1c, C1c, F1c, V2c, E2c, I2c, A2c, R2c, C2c, F2c, Ic, Cc, Fc = S1c.tolist(), E1c.tolist(), I1c.tolist(), A1c.tolist(), R1c.tolist(), C1c.tolist(), F1c.tolist(), V2c.tolist(), E2c.tolist(), I2c.tolist(), A2c.tolist(), R2c.tolist(), C2c.tolist(), F2c.tolist(), Ic.tolist(), Cc.tolist(), Fc.tolist()

                seiar_results_with_migration3.append({"Province": index_to_province[idx],'goal':goal,'n1s':n1s,'n2s':n2s, 'n3s':n3s,'Ia':Ia,'Ca' :Ca, 'Fa':Fa,'Ib':Ib,'Cb' :Cb, 'Fb':Fb, 'Ic':Ic,'Cc' :Cc, 'Fc':Fc})

            infection_df = pd.DataFrame()

            for i in range(31):
                choose_number = i
                example_province_seiar_migration = seiar_results_with_migration3[choose_number]
                province_name = index_to_province[choose_number].split('省')[0]

                y1 = [sum(x) for x in zip(seiar_results_with_migration3[i]["Ia"], seiar_results_with_migration3[i]["Ib"], seiar_results_with_migration3[i]["Ic"])]
                diff_y1 = [y1[i] - y1[i-1] for i in range(1, len(y1))]
                diff_y1 = [y1[0]] + diff_y1
                date_range = pd.date_range(start='2023-07-31', end='2024-03-03', freq='D')

                y2 = [sum(x) for x in zip(seiar_results_with_migration3[i]["Ca"], seiar_results_with_migration3[i]["Cb"], seiar_results_with_migration3[i]["Cc"])]
                diff_y2 = [y2[i] - y2[i-1] for i in range(1, len(y2))]
                diff_y2 = [y2[0]] + diff_y2


                y3 = [sum(x) for x in zip(seiar_results_with_migration3[i]["Fa"], seiar_results_with_migration3[i]["Fb"], seiar_results_with_migration3[i]["Fc"])]
                diff_y3 = [y3[i] - y3[i-1] for i in range(1, len(y3))]
                diff_y3 = [y3[0]] + diff_y3

                for j in range(len(date_range)):
                    case = diff_y1[j]
                    zz = diff_y2[j]
                    death = diff_y3[j]

                    
                    infection_df = pd.concat([infection_df, pd.DataFrame({'date': date_range[j], 'province': province_name, '感染': case, '重症':zz,'死亡':death}, index=[0])], ignore_index=True)

            infection_df.to_excel(f"D:\\Research\\Project\\PROJECTS\\其他\\Flu\\延迟{vt}天_目标疫苗覆盖率{goal*100}%.xlsx")

## 6. Provincial difference simulation

In [ ]:
seiar_results_with_migration = []
for rx in range(0,91):
    for idx, (beta1,Na, Nb,Nc, I0xa, I0xb, I0xc,n1,n2,n3) in enumerate(zip(beta1s,Nas,Nbs,Ncs, I0as,I0bs,I0cs,n1s,n2s,n3s)):
        if idx in [8, 12, 13, 15, 18, 19, 20, 22, 25, 27]:
            n1 =  n1+ rx*0.01 
            n2 =  n2 + rx*0.01 
            n3 =  n3 + rx*0.01 
        if idx in [8, 12, 13, 15, 18, 19, 20, 22, 25, 27]:
            n1 =  n1+ rx*0.005 
            n2 =  n2 + rx*0.005 
            n3 =  n3 + rx*0.005 
        else:
            n1 = n1+ rx*0.001
            n2 = n2+ rx*0.001
            n3 =  n3+ rx*0.001

        beta2 = beta1*0.94 
        beta3 = beta1*0.55
        beta12 = beta1*0.9
        beta32 =  beta1*0.5
        beta21 = beta1*0.9
        beta31 = beta1*0.5
        beta13 = beta1*0.9
        beta23 = beta1*0.9


        E1a0, I1a0, A1a0, R1a0, C1a0, F1a0, E2a0, I2a0, A2a0, R2a0, C2a0, F2a0 = 0, I0xa*(1-n1), 0, 0 , 0, 0 ,0, I0xa*n1, 0, 0 , 0, 0   
        I0a = I0xa
        C0a = 0
        F0a = 0
        S1a0 = Na*(1-n1) - E1a0 - I1a0 - A1a0 - R1a0 - C1a0 - F1a0
        V2a0 = Na*n1 - E2a0 - I2a0 - A2a0 - R2a0 - C2a0 - F2a0

        #Group b
        E1b0, I1b0, A1b0, R1b0, C1b0, F1b0, E2b0, I2b0, A2b0, R2b0, C2b0, F2b0 = 0, I0xb*(1-n2), 0, 0 , 0, 0 ,0, I0xb*n2, 0, 0 , 0, 0
        I0b = I0xb
        C0b = 0
        F0b = 0
        S1b0 = Nb*(1-n2) - E1b0 - I1b0 - A1b0 - R1b0 - C1b0 - F1b0
        V2b0 = Nb*n2 - E2b0 - I2b0 - A2b0 - R2b0 - C2b0 - F2b0

        #Group c
        E1c0, I1c0, A1c0, R1c0, C1c0, F1c0, E2c0, I2c0, A2c0, R2c0, C2c0, F2c0 = 0, I0xc*(1-n3), 0, 0 , 0, 0 ,0, I0xc*n3, 0, 0 , 0, 0
        I0c = I0xc
        C0c = 0
        F0c = 0
        S1c0 = Nc*(1-n3) - E1c0 - I1c0 - A1c0 - R1c0 - C1c0 - F1c0
        V2c0 = Nc*n3 - E2c0 - I2c0 - A2c0 - R2c0 - C2c0 - F2c0

        y0 =  S1a0, E1a0, I1a0, A1a0, R1a0, C1a0, F1a0, V2a0, E2a0, I2a0, A2a0, R2a0, C2a0, F2a0, I0a, C0a, F0a,  \
              S1b0, E1b0, I1b0, A1b0, R1b0, C1b0, F1b0, V2b0, E2b0, I2b0, A2b0, R2b0, C2b0, F2b0, I0b, C0b, F0b,  \
              S1c0, E1c0, I1c0, A1c0, R1c0, C1c0, F1c0, V2c0, E2c0, I2c0, A2c0, R2c0, C2c0, F2c0, I0c, C0c, F0c
        t = np.linspace(0, days, days+1) 

        def X(t,vt):
            if t < vt: 
                return 0, 0, 0
            else:
                return 0, 0, 0
    
        def get_daily_migration(day, idx):
            if day < days:
                return daily_inflow[int(day), idx], daily_outflow[int(day), idx]
            else:
                return 0, 0  

        ret = odeint(lambda y, t: flu_model_with_migration(y, t, Na,Nb,Nc, beta1, beta2,beta3,beta12, beta21,beta31,beta13,beta32,beta23,
            sigma, delta, gamma_A, gamma_I, kappa, p, q, f, n1,n2,n3, VEI1,VES1, VEC1,VEF1,VEI2,VES2, VEC2,VEF2,VEI3,VES3, VEC3,VEF3,
            *get_daily_migration(t, idx),*X(t,vt) ),y0, t)



        S1a, E1a, I1a, A1a, R1a, C1a, F1a, V2a, E2a, I2a, A2a, R2a, C2a, F2a, Ia, Ca, Fa, \
        S1b, E1b, I1b, A1b, R1b, C1b, F1b, V2b, E2b, I2b, A2b, R2b, C2b, F2b, Ib, Cb, Fb, \
        S1c, E1c, I1c, A1c, R1c, C1c, F1c, V2c, E2c, I2c, A2c, R2c, C2c, F2c, Ic, Cc, Fc = ret.T


        S1a, E1a, I1a, A1a, R1a, C1a, F1a, V2a, E2a, I2a, A2a, R2a, C2a, F2a, Ia, Ca, Fa = S1a.tolist(), E1a.tolist(), I1a.tolist(), A1a.tolist(), R1a.tolist(), C1a.tolist(), F1a.tolist(), V2a.tolist(), E2a.tolist(), I2a.tolist(), A2a.tolist(), R2a.tolist(), C2a.tolist(), F2a.tolist(), Ia.tolist(), Ca.tolist(), Fa.tolist()
        S1b, E1b, I1b, A1b, R1b, C1b, F1b, V2b, E2b, I2b, A2b, R2b, C2b, F2b, Ib, Cb, Fb = S1b.tolist(), E1b.tolist(), I1b.tolist(), A1b.tolist(), R1b.tolist(), C1b.tolist(), F1b.tolist(), V2b.tolist(), E2b.tolist(), I2b.tolist(), A2b.tolist(), R2b.tolist(), C2b.tolist(), F2b.tolist(), Ib.tolist(), Cb.tolist(), Fb.tolist()
        S1c, E1c, I1c, A1c, R1c, C1c, F1c, V2c, E2c, I2c, A2c, R2c, C2c, F2c, Ic, Cc, Fc = S1c.tolist(), E1c.tolist(), I1c.tolist(), A1c.tolist(), R1c.tolist(), C1c.tolist(), F1c.tolist(), V2c.tolist(), E2c.tolist(), I2c.tolist(), A2c.tolist(), R2c.tolist(), C2c.tolist(), F2c.tolist(), Ic.tolist(), Cc.tolist(), Fc.tolist()
        Ia_sum = Ia[-1]
        Ib_sum = Ib[-1]
        Ic_sum = Ic[-1]

        Ca_sum = Ca[-1]
        Cb_sum = Cb[-1]
        Cc_sum = Cc[-1]

        Fa_sum = Fa[-1]
        Fb_sum = Fb[-1]
        Fc_sum = Fc[-1]

        seiar_results_with_migration.append({'rx':rx,'n1':n1,'n2':n2,'n3':n3,'Province': index_to_province[idx],'未成年人累积病例数':Ia_sum,'中年人累积病例数' :Ib_sum, '老年人累积病例数':Ic_sum,
                                             '未成年人累积重症数':Ca_sum,'中年人累积重症数' :Cb_sum, '老年人累积重症数':Cc_sum,
                                             '未成年人累积死亡数':Fa_sum,'中年人累积死亡数' :Fb_sum, '老年人累积死亡数':Fc_sum,})
        #seiar_results_with_migration.append({"Province": index_to_province[idx], 'n1':n1,'n2':n2,'n3':n3,'Ia':Ia[-1],'Ca' :Ca[-1], 'Fa':Fa[-1],'Ib':Ib[-1],'Cb' :Cb, 'Fb':Fb[-1],'Ic':Ic[-1],'Cc' :Cc[-1], 'Fc':Fc[-1]})
res_df = pd.DataFrame(seiar_results_with_migration)
res_df.to_excel('D:\\Research\\Project\\PROJECTS\\其他\\Flu\\地区1.xlsx')